# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Lee  ->  K. Lee  |  ['K. Lee']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
H. Linz  ->  H. Linz  |  ['H. Linz']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
M. Kürster  ->  M. Kürster  |  ['M. Kürster']
M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
J. Li  ->  J. Li  |  ['J. Li']
M. Smith  ->  M. Smith  |  ['M. Smith']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 90 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.13740


extracting tarball to tmp_2310.13740...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.14061


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.13740/Planck.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

Found 75 bibliographic references in tmp_2310.14716/transmission.bbl.
Retrieving document from  https://arxiv.org/e-print/2310.14723


extracting tarball to tmp_2310.14723...

 done.
Retrieving document from  https://arxiv.org/e-print/2310.14874


extracting tarball to tmp_2310.14874...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.14715-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.14715) | **The CARMENES search for exoplanets around M dwarfs. Telluric absorption  corrected high S/N optical and near-infrared template spectra of 382 M dwarf  stars**  |
|| E. Nagel, et al. -- incl., <mark>M. Kürster</mark> |
|*Appeared on*| *2023-10-24*|
|*Comments*| *31 pages, 24 figures, 3 tables, accepted for publication by A&A*|
|**Abstract**| Light from celestial objects interacts with the molecules of the Earth's atmosphere, resulting in the production of telluric absorption lines in ground-based spectral data. Correcting for these lines, which strongly affect red and infrared wavelengths, is often needed in a wide variety of scientific applications. Here, we present the template division telluric modeling (TDTM) technique, a method for accurately removing telluric absorption lines in stars that exhibit numerous intrinsic features. Based on the Earth's barycentric motion throughout the year, our approach is suited for disentangling telluric and stellar spectral components. By fitting a synthetic transmission model, telluric-free spectra are derived. We demonstrate the performance of the TDTM technique in correcting telluric contamination using a high-resolution optical spectral time series of the feature-rich M3.0 dwarf star Wolf 294 that was obtained with the CARMENES spectrograph. We apply the TDTM approach to the CARMENES survey sample, which consists of 382 targets encompassing 22357 optical and 20314 near-infrared spectra, to correct for telluric absorption. The corrected spectra are coadded to construct template spectra for each of our targets. This library of telluric-free, high signal-to-noise ratio, high-resolution (R>80000) templates comprises the most comprehensive collection of spectral M-dwarf data available to date, both in terms of quantity and quality, and is available at the project website (this http URL). |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.14716-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.14716) | **Euclid preparation. XXXI. The effect of the variations in photometric  passbands on photometric-redshift accuracy**  |
|| E. Collaboration, et al. -- incl., <mark>M. Schirmer</mark> |
|*Appeared on*| *2023-10-24*|
|*Comments*| *19 pages, 13 figures; Accepted for publication in A&A*|
|**Abstract**| The technique of photometric redshifts has become essential for the exploitation of multi-band extragalactic surveys. While the requirements on photo-zs for the study of galaxy evolution mostly pertain to the precision and to the fraction of outliers, the most stringent requirement in their use in cosmology is on the accuracy, with a level of bias at the sub-percent level for the Euclid cosmology mission. A separate, and challenging, calibration process is needed to control the bias at this level of accuracy. The bias in photo-zs has several distinct origins that may not always be easily overcome. We identify here one source of bias linked to the spatial or time variability of the passbands used to determine the photometric colours of galaxies. We first quantified the effect as observed on several well-known photometric cameras, and found in particular that, due to the properties of optical filters, the redshifts of off-axis sources are usually overestimated. We show using simple simulations that the detailed and complex changes in the shape can be mostly ignored and that it is sufficient to know the mean wavelength of the passbands of each photometric observation to correct almost exactly for this bias; the key point is that this mean wavelength is independent of the spectral energy distribution of the source}. We use this property to propose a correction that can be computationally efficiently implemented in some photo-z algorithms, in particular template-fitting. We verified that our algorithm, implemented in the new photo-z code Phosphoros, can effectively reduce the bias in photo-zs on real data using the CFHTLS T007 survey, with an average measured bias Delta z over the redshift range 0.4<z<0.7 decreasing by about 0.02, specifically from Delta z~0.04 to Delta z~0.02 around z=0.5. Our algorithm is also able to produce corrected photometry for other applications. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.14061-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.14061) | **Investigating Protostellar Accretion across the mass spectrum with the  JWST: discovery of a collimated jet from the low luminosity protostar IRAS  16253-2429 in a quiescent accretion phase**  |
|| M. Narang, et al. -- incl., <mark>H. Beuther</mark>, <mark>H. Linz</mark> |
|*Appeared on*| *2023-10-24*|
|*Comments*| *To be submitted to ApJL. Comments and feedback welcome*|
|**Abstract**| Investigating Protostellar Accretion (IPA) is a JWST Cycle 1 GO program that used NIRSpec IFU and MIRI MRS to obtain 2.9--28~$\mu$m spectral cubes of young, deeply embedded, protostars with luminosities of 0.2 to 10,000~L$_{\odot}$ and central masses of 0.15 to 12~M$_{\odot}$. In this Letter, we report the discovery of a highly collimated atomic jet from the Class 0 protostar IRAS~16253-2429, the lowest luminosity (mass) source in the IPA program. The collimated jet is clearly detected in multiple [Fe~II] lines, [Ne~II], [Ni~II], and H~I lines, but not in molecular emission. From our analysis, we find that the atomic jet has a velocity of about 169~$\pm$~15~km\,s$^{-1}$, after correcting for inclination. The width of the jet increases with distance from the central protostar from 23 to~60 au, corresponding to an opening angle of 2.6~$\pm$~0.5\arcdeg. By comparing the measured flux ratios of various fine structure lines to those predicted by simple shock models, we derive a shock velocity of 54~km\,s$^{-1}$ and a preshock density of 2.0$\times10^{3}$~cm$^{-3}$ at the base of the jet. From these quantities, and assuming a cylindrical cross-section for the jet, we derive an upper limit for the mass loss rate from the protostar of 1.1$\times10^{-10}~M_{\odot}$~yr~$^{-1}$. The low mass loss rate is consistent with simultaneous measurements of low mass accretion rate ($\sim 1.3 \times 10^{-9}~M_{\odot}$~yr~$^{-1}$) for IRAS~16253-2429 from JWST observations (Watson et al. in prep), indicating that the protostar is in a quiescent accretion phase. Our results demonstrate that very low-mass protostars can drive highly collimated, atomic jets, even during the quiescent phase. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.14874-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.14874) | **The Cosmic Dust Storms in SNIa-CSM 2018evt**  |
|| L. Wang, et al. -- incl., <mark>M. Smith</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2023-10-24*|
|*Comments*| *Resubmitted to Nature Astronomy, 6 main figures, 7 extended figures, and 2 extended tables*|
|**Abstract**| We report evidence of dust formation in the cold, dense shell behind the ejecta-circumstellar medium (CSM) interaction in the Type Ia (SNIa) SN2018evt three years after the explosion, characterized by a rise in the mid-infrared (MIR) flux accompanied by an accelerated decline in the optical. Such a dust-formation picture is also corroborated by the concurrent evolution of the profiles of the Ha emission lines. Our models suggest enhanced dust concentration at increasing distances from the SN as compared to what can be expected from the density profile of the mass loss from a steady stellar wind. This may indicate an enhanced dust presence at larger distances from the progenitor star. The dust distribution can be modeled in terms of a double-shell which assumes a sudden change of the density profile of the dust or a single-shell model with a flatter radial profile. The inferred mass-loss history of SN2018evt is consistent with a progenitor's mass loss in a binary system containing a C/O white dwarf and a massive asymptotic giant branch star. The grand rebrightening in the MIR after day +310 is attributed to the formation of new dust within the CDS region behind the forward shock. The mass of the newly-formed dust increases rapidly with time and follows a power law with an index of 4. By the time of the last MIR observations at day +1041, a total amount of 1.2+-0.2 x 10^{-2} Msun of dust has been produced, making SN 2018evt one of the most prolific dust factory among SNe with evidence of dust formations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.13740-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.13740) | **Cosmoglobe DR1. III. First full-sky model of polarized synchrotron  emission from all WMAP and Planck LFI data**  |
|| D. J. Watts, et al. -- incl., <mark>K. Lee</mark> |
|*Appeared on*| *2023-10-24*|
|*Comments*| *15 pages, 15 figures, submitted to A&A*|
|**Abstract**| We present the first model of full-sky polarized synchrotron emission that is derived from all WMAP and Planck LFI frequency maps. The basis of this analysis is the set of end-to-end reprocessed Cosmoglobe Data Release 1 sky maps presented in a companion paper, which have significantly lower instrumental systematics than the legacy products from each experiment. We find that the resulting polarized synchrotron amplitude map has an average noise rms of $3.2\,\mathrm{\mu K}$ at 30 GHz and $2^{\circ}$ FWHM, which is 30% lower than the recently released BeyondPlanck model that included only LFI+WMAP Ka-V data, and 29% lower than the WMAP K-band map alone. The mean $B$-to-$E$ power spectrum ratio is $0.40\pm0.02$, with amplitudes consistent with those measured previously by Planck and QUIJOTE. Assuming a power law model for the synchrotron spectral energy distribution, and using the $T$--$T$ plot method, we find a full-sky inverse noise-variance weighted mean of $\beta_{\mathrm{s}}=-3.07\pm0.07$ between Cosmoglobe DR1 K-band and 30 GHz, in good agreement with previous estimates. In summary, the novel Cosmoglobe DR1 synchrotron model is both more sensitive and systematically cleaner than similar previous models, and it has a more complete error description that is defined by a set of Monte Carlo posterior samples. We believe that these products are preferable over previous Planck and WMAP products for all synchrotron-related scientific applications, including simulation, forecasting and component separation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.14447-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.14447) | **SOUL at LBT: commissioning results, science and future**  |
|| E. Pinna, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-10-24*|
|*Comments*| *13 pages, 10 figures, Adaptive Optics for Extremely Large Telescopes 7th Edition, 25-30 Jun 2023 Avignon (France)*|
|**Abstract**| The SOUL systems at the Large Bincoular Telescope can be seen such as precursor for the ELT SCAO systems, combining together key technologies such as EMCCD, Pyramid WFS and adaptive telescopes. After the first light of the first upgraded system on September 2018, going through COVID and technical stops, we now have all the 4 systems working on-sky. Here, we report about some key control improvements and the system performance characterized during the commissioning. The upgrade allows us to correct more modes (500) in the bright end and increases the sky coverage providing SR(K)>20% with reference stars G$_{RP}$<17, opening to extragalcatic targets with NGS systems. Finally, we review the first astrophysical results, looking forward to the next generation instruments (SHARK-NIR, SHARK-Vis and iLocater), to be fed by the SOUL AO correction. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.14530-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.14530) | **Mock Observations: Formation and Evolution of diffuse light in Galaxy  Groups and Clusters in the IllustrisTNG Simulations**  |
|| L. Tang, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-10-24*|
|*Comments*| *8 figures, 2 tables, Accepted for publication in ApJ*|
|**Abstract**| In this paper, by analyzing mock images from the IllustrisTNG100-1 simulation, we examine the properties of the diffuse light and compare them to those of central and satellite galaxies. Our findings suggest that the majority of the diffuse light originates from satellites. This claim is supported by the similarity between the age and metallicity distributions of the diffuse light and those of the satellites. Notably, the color distribution of the diffuse light gradually evolves to resemble that of the centrals at lower redshifts, suggesting a coevolution or passive process. The radial profiles of the diffuse light reveal distinct trends, with the inner regions displaying a relatively flat distribution and the outer regions showing a descending pattern. This finding suggests that the formation of the diffuse light is influenced by both major mergers and stellar tidal stripping. Moreover, strong correlations are found between the stellar mass of the diffuse light and the overall stellar mass of the satellites, as well as between the stellar mass of the diffuse light and the number of satellites within groups or clusters. These relationships can be described by power-law and logarithmic functions. Overall, the diffuse light components predominantly originate from satellites with intermediate ages and metallicities. These satellites typically fall within the stellar mass range of $\rm 8<\log_{10}M_{star}/M_{\odot}< 10$ and the color range of $\rm -1<[g-r]^{0.1}< 0$. As the redshift decreases, the growth of the diffuse light is primarily influenced by the redder satellites, while the most massive and reddest satellites have minimal roles in its growth. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.14723-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.14723) | **Testing the spatial geometry of the universe with TianQin: the prospect  of using supermassive black hole binaries**  |
|| Y. Pan, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2023-10-24*|
|*Comments*| *This article has been accepted by Astronomy & Astrophysics*|
|**Abstract**| The determination of the spatial geometry of the universe plays an important role in modern cosmology. Any deviation from the cosmic curvature $\Omega_K=0$ would have a profound impact on the primordial inflation paradigm and fundamental physics. In this paper, we carry out a systematic study of the prospect of measuring cosmic curvature with the inspiral signal of supermassive black hole binaries (SMBHBs) that could be detected with TianQin. The study is based on a cosmological-model-independent method that extended the application of gravitational wave (GW) standard sirens in cosmology. By comparing the distances from future simulated GW events and simulated $H(z)$ data, we evaluate if TianQin would produce robust constraints on the cosmic curvature parameter $\Omega_{k}$. More specifically, we consider 3-yr to 10-yr observations of supermassive black hole binaries with total masses ranging from $10^{3}M_\odot$ to $10^{7}M_\odot$. Our results show that in the future, with the synergy of 10-yr high-quality observations, we can tightly constrain the curvature parameter at the level of $1\sigma$ $\Omega_k=-0.002\pm0.061$. Moreover, our findings indicate that the total mass of SMBHB does influence the estimation of cosmic curvature, implied by the analysis performed on different subsamples of gravitational wave data. Therefore, TianQin is expected to provide a powerful and competitive probe of the spatial geometry of the universe, compared to future spaced-based detectors such as DECIGO. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.14715.md
    + _build/html/tmp_2310.14715/./Figures/accuracy_O2.png
    + _build/html/tmp_2310.14715/./Figures/tdtm_vis_H2O.png
    + _build/html/tmp_2310.14715/./Figures/residuals-vs-linedepth.png
    + _build/html/tmp_2310.14715/./Figures/residuals-vs-airmass.png
    + _build/html/tmp_2310.14715/./Figures/residuals-vs-water_content.png
exported in  _build/html/2310.14716.md
    + _build/html/tmp_2310.14716/./figs/bias_delta.png
    + _build/html/tmp_2310.14716/./figs/shift_bias.png
    + _build/html/tmp_2310.14716/./figs/corr_factor.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.14715-b31b1b.svg)](https://arxiv.org/abs/2310.14715)<mark>Appeared on: 2023-10-24</mark> -  _31 pages, 24 figures, 3 tables, accepted for publication by A&A_

</div>
<div id="authors">

E. Nagel, et al. -- incl., <mark>M. Kürster</mark>

</div>
<div id="abstract">

**Abstract:** Light from celestial objects interacts with the molecules of the Earth's atmosphere,resulting in the production of telluric absorption lines in ground-based spectral data.Correcting for these lines, which strongly affect red and infrared wavelengths,is often needed in a wide variety of scientific applications.Here, we present the template division telluric modeling (TDTM) technique,a method for accurately removing telluric absorption lines in stars thatexhibit numerous intrinsic features.Based on the Earth's barycentric motionthroughout the year, our approach is suited for disentangling telluric andstellar spectral components.By fitting a synthetic transmission model, telluric-free spectra are derived.We demonstrate the performance of the TDTM technique in correctingtelluric contaminationusing a high-resolution optical spectral time series of thefeature-rich M3.0 dwarf star Wolf 294 that was obtained with the CARMENES spectrograph.We apply the TDTM approach to the CARMENES survey sample, which consists of382 targets encompassing 22 357 optical and 20 314 near-infrared spectra,to correct for telluric absorption.The corrected spectra are coadded to constructtemplate spectra for each of our targets.This library of telluric-free, high signal-to-noise ratio, high-resolution ( $\mathcal{R}>80 000$ ) templates comprisesthe most comprehensive collection of spectralM-dwarf data available to date, both in terms of quantity and quality,and is available at the project website.

</div>

<div id="div_fig1">

<img src="tmp_2310.14715/./Figures/accuracy_O2.png" alt="Fig12" width="100%"/>

**Figure 12. -** 
            Example of telluric line fits for the $O_2$ band in a single Wolf 294 spectrum.
            The top panel displays the residual telluric spectrum (black dots)
            after dividing the science spectrum by the stellar template,
            and the best-fit telluric model (green dots) derived with \texttt{molecfit}.
            The subplots illustrate the individual telluric lines (top) and
            the residuals (bottom, black dots), with dashed red lines marking
            a $2 \%$ deviation between data and model. The standard deviation $\sigma$
            of the residuals is also shown in each subplot.
             (*figure:accuracy_O2*)

</div>
<div id="div_fig2">

<img src="tmp_2310.14715/./Figures/tdtm_vis_H2O.png" alt="Fig7" width="100%"/>

**Figure 7. -** 
Illustration of the TDTM method.
_Panel a:_ Segment of the VIS template spectrum of Wolf 294.
The number $M_k$ of exposure pixels that contribute to each template knot is color-coded.
The red shaded wavelength ranges mark knots with $M_k = 0$.
_Panel b:_
        One residual telluric spectrum $F_{n,i} / S(\lambda_{n,i})$(black dots)
after the division of the science spectrum by the template,
and the best-fit telluric model ($T(\lambda_{n,i})$, green line) derived with \texttt{molecfit}.
The red shaded wavelength ranges are excluded from the transmission model fit.
_Panel c:_
	Absolute residuals $F_{n,i}/S(\lambda_{n,i}) - T(\lambda_{n,i})$ of the fit.
_Panel d:_
CARMENES spectrum before ($F_{n,i}$, red line) and after ($F_{n,i} / T(\lambda_{n,i})$)
correction with the transmission model derived with \texttt{molecfit}(black line).
_Panel e:_
	Telluric free high S/N template spectrum of Wolf 294 (black line) built using 444 telluric absorption-corrected
CARMENES observations. The order has an S/N of 2310.
 (*figure:tdtm_vis*)

</div>
<div id="div_fig3">

<img src="tmp_2310.14715/./Figures/residuals-vs-linedepth.png" alt="Fig3.1" width="33%"/><img src="tmp_2310.14715/./Figures/residuals-vs-airmass.png" alt="Fig3.2" width="33%"/><img src="tmp_2310.14715/./Figures/residuals-vs-water_content.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** 
        Metrics for assessing the quality of the telluric correction.
        _Top panel:_
        Residual standard deviation ($\sigma$) vs. line depth for a single Wolf 294 VIS (blue) and NIR (red) spectrum.
        _Middle panel:_ Median residual standard deviation ($\tilde{\sigma}$) vs. airmass for all Wolf 294 VIS and NIR spectra.
        _Bottom panel:_ Median $\tilde{\sigma}$ for telluric water bands vs. precipitable water vapor for all Wolf 294 VIS and NIR spectra. (*figure:residuals-vs-linedepth*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.14715"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\rom}[1]{\uppercase\expandafter{\romannumeral #1\relax}}$
$\newcommand{\phz}{photometric redshift}$
$\newcommand{\phdz}{photometric-redshift}$
$\newcommand{\dd}{\mathrm{d}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ preparation. XXXI. The effect of the variations in photometric passbands on photometric-redshift accuracy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.14716-b31b1b.svg)](https://arxiv.org/abs/2310.14716)<mark>Appeared on: 2023-10-24</mark> -  _19 pages, 13 figures; Accepted for publication in A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>M. Schirmer</mark>

</div>
<div id="abstract">

**Abstract:** The technique of photometric redshifts has become essential for the exploitation of multi-band extragalactic surveys. While the requirements on $\phz$ s for the study of galaxy evolution mostly pertain to the precision and to the fraction of outliers, the most stringent requirement ${in}$ their use in cosmology is on the accuracy, with a level of bias at the sub-percent level for the $\Euclid$ cosmology mission. A separate, and challenging, calibration process is needed to control the bias at this level of accuracy. ${The bias in \phz s has several distinct origins that may not always be easily overcome.}$ We identify here one source of bias linked to the spatial or time variability of the passbands used to determine the photometric colours of galaxies. We first quantified the effect as observed on several well-known photometric cameras, and ${found in particular that, due to the properties of optical filters, the redshifts of off-axis sources are usually overestimated}$ . We show using simple simulations that the detailed and complex changes in the shape can be mostly ignored and that it is sufficient to know the ${mean}$ wavelength of the passbands of each photometric observation to correct almost exactly for this bias; ${the key point is that this mean wavelength is independent of the spectral energy distribution of the source}$ . We use this property to propose a ${correction that can be computationally efficiently implemented in some \phdz algorithms}$ , in particular template-fitting. We verified that our algorithm, implemented in the new $\phdz$ code \texttt{Phosphoros} , can effectively reduce the bias in $\phz$ s on real data using the CFHTLS T007 survey, with an average measured bias $\Delta z$ over the redshift range $0.4\le z\le 0.7$ decreasing by about 0.02, ${specifically from $\Delta z\simeq 0.04$ to $\Delta z\simeq 0.02$ around $z=0.5$}$ . Our algorithm is also able to produce corrected photometry for other applications.

</div>

<div id="div_fig1">

<img src="tmp_2310.14716/./figs/bias_delta.png" alt="Fig2" width="100%"/>

**Figure 2. -** Bias $\Delta z$ resulting from a shift of $\Delta\mu$ in the $G$ passband for $z=0.15$(solid blue line). The dashed red line is the `theoretical' bias $\Delta\mu/400$ nm, where $\Delta\mu$ is the error on the location of the Balmer break. The grey area shows the domain of variations of the mean wavelengths of the measured MegaCam $r$ passbands (from 0 to 8 nm). (*fig:toy_mean*)

</div>
<div id="div_fig2">

<img src="tmp_2310.14716/./figs/shift_bias.png" alt="Fig4" width="100%"/>

**Figure 4. -** Median bias in $\phdz$ predictions as a function of redshift for the original catalogue (black line) and when taking into account the correction in the mean passband wavelengths (red line). {The dashed blue line shows the reduction in the bias between the two solid curves.} The grey area indicates the \Euclid requirement. While it is clear that the passband correction reduces the bias significantly, there is still a need for a post-processing calibration step in order to meet the \Euclid requirements. (*fig:shift_bias*)

</div>
<div id="div_fig3">

<img src="tmp_2310.14716/./figs/corr_factor.png" alt="Fig5" width="100%"/>

**Figure 5. -** Correction factors $C^{\alpha}_T(\Delta\lambda)$(blue points) and the corresponding approximation with a second-degree polynomial (red line) for four arbitrary values of the passband $T(\lambda)$ and of the coordinates $\alpha$ in the model grid. (*fig:corr_factor*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.14716"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

306  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
